In [2]:
import pandas as pd

In [3]:
excel_sheet = pd.read_excel("Datos TallerMCII 2023-1 (1).xlsx", sheet_name = "DATOS PUNTO I")

2. Considere el número mensual de pernoctaciones desde el extranjero a Europa (en millones).
 - a) Utilice la técnica de aditivo estacional y multiplicativo estacional para pronosticar el número de pernoctaciones que se esperan en los próximos 3 meses, con su respectivo margen de error. Determine si los residuos de cada modelo son ruido blanco y seleccione el mejor. Solamente realice los pronósticos con el mejor modelo. Considere 8000 simulaciones y 600 repeticiones. Interprete.
 - b) Utilice la técnica de regresión múltiple con variables ficticias para pronosticar el número de pernoctaciones que se esperan en los próximos 3 meses. Determine si los residuos del modelo son ruido blanco.

In [4]:
pernotaciones_frame = excel_sheet.iloc[:,6:8]
pernotaciones_frame["Periodo"] = pd.to_datetime(pernotaciones_frame["Periodo"], format = "%Y-%m" )

In [5]:
pernotaciones_frame.shape

(569, 2)

In [6]:
pernotaciones_frame = pernotaciones_frame.dropna()

In [7]:
pernotaciones_frame.shape

(200, 2)

In [8]:
pernotaciones_frame.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column                                                         Non-Null Count  Dtype         
---  ------                                                         --------------  -----         
 0   Periodo                                                        200 non-null    datetime64[ns]
 1   Número mensual de pernoctaciones desde el extranjero a Europa  200 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 4.7 KB


In [9]:
pernotaciones_frame.columns = ["Periodo", "Pernotaciones"]
pernotaciones_frame.set_index("Periodo", inplace = True)
pernotaciones_frame.describe()

,Pernotaciones
count,200.000000
mean,1688.982589
std,425.914580
min,811.080000
25%,1406.153750
50%,1611.952500
75%,1913.162250
max,2601.068000


In [10]:
pernotaciones_frame.resample("M").first()

,Pernotaciones
Periodo,
2006-06-30,1462.712000
2006-07-31,1618.582000
2006-08-31,1867.108000
2006-09-30,1163.087000
2006-10-31,1247.724000
...,...
2022-09-30,1124.965249
2022-10-31,1654.963000
2022-11-30,2601.068000


In [11]:

import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing



In [12]:
result_multiplicative = seasonal_decompose(pernotaciones_frame['Pernotaciones'], model='multiplicative')
result_additive = seasonal_decompose(pernotaciones_frame['Pernotaciones'], model='additive')


In [13]:
trend_multiplicative = result_multiplicative.trend
seasonal_multiplicative = result_multiplicative.seasonal
residuals_multiplicative = result_multiplicative.resid

trend_additive = result_additive.trend
seasonal_additive = result_additive.seasonal
residuals_additive = result_additive.resid


In [14]:
# Create the Holt-Winters model with alpha=0 (no trend smoothing)
model = ExponentialSmoothing(pernotaciones_frame['Pernotaciones'], seasonal_periods=12)

# Fit the model to the training data
fitted_model = model.fit()

# Forecast the next 3 months
forecast = fitted_model.forecast(3)

# Print the forecasted values
print("Forecasted values:", forecast) 

Forecasted values: 2023-02-01    2123.707736
2023-03-01    2123.707736
2023-04-01    2123.707736
Freq: MS, dtype: float64


/workspaces/Metodos-cuantitativos/env/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


In [15]:
model